## Chapter 5. Array-Based Sequences - Notes

P.183

### 5.1 Python's Sequence Types

In this chapter we explore Python's built-in sequence classes like __list__, __tuple__, and __str__. Each of them support indexing to access individual elements of a sequence, using syntax seq[k], and each uses low-level concept known as an __array__ to represent sequence.

### 5.2 Low-Level Arrays

Any individual byte of memory can be stored and retrieved in $O(1)$ time. In general, a programming language keeps track of the association between an identifier and the memory address in which associated value is stored. A group of related variables can be stored in a contiguous portion of the computer's memory. We will denote such a representation as an __array__. Each location - __cell__, location within array - __index__.

### 5.2.1 Referential Arrays

Python represents a list or tuple instance using internal storage mechanism of an array of object __references__. A single list instance may include multiple references to the same object as elements of the list, and it is possible for a single object to be an element of two or more lists, as those lists simply store references back to the object. 

__[Side Note](https://medium.com/@meghamohan/mutable-and-immutable-side-of-python-c2145cf72747):__ Objects of built-in types like (int, float, bool, str, tuple, unicode) are immutable. Objects of built-in types like (list, set, dict) are mutable. 

### 5.2.2 Compact Arrays in Python

The strings are represented using an array of characters (not an array of references). We refer to this type representation as  a __compact array__. The overall memory usage for a compact structure is much lower.

Constructor for the array class requires a __type code__ as a first parameter ([More Info](https://docs.python.org/3/library/array.html)) 

__Ex__: The type code 'i' designates an array of (signed) integers, typically represented using at least 16-bits each:

In [1]:
from array import array

primes = array('i', [2, 3, 5, 7, 11, 13, 17, 19])
primes

array('i', [2, 3, 5, 7, 11, 13, 17, 19])

In [2]:
[i for i in primes]

[2, 3, 5, 7, 11, 13, 17, 19]

### 5.3 Dynamic Arrays and Amortization

When creating a low level array in computer system, the precise size of that array must be explicitly declared in order for the system to properly allocate a consecutive piece of memory for its storage (Pythons _tuple_ and _str_). List, on contrary, uses __dynamic array__, which allows the addition of element, with no apparent limit on the overall capacity of the list. Every time the capacity reached, the class requests more memory.

__Ex:__ An experiment to explore the relationship between a list's length and its underlying size in Python. We use __getsizeof__ from [_sys_](https://docs.python.org/3/library/sys.html) module that reports number of bytes that are being used to store an object in Python:

In [3]:
import sys

data = []

for k in range(20):
    a = len(data)
    b = sys.getsizeof(data)
    print('Length: {0:3d}; Size in bytes: {1:4d}'.format(a, b))
    data.append(None)

Length:   0; Size in bytes:   64
Length:   1; Size in bytes:   96
Length:   2; Size in bytes:   96
Length:   3; Size in bytes:   96
Length:   4; Size in bytes:   96
Length:   5; Size in bytes:  128
Length:   6; Size in bytes:  128
Length:   7; Size in bytes:  128
Length:   8; Size in bytes:  128
Length:   9; Size in bytes:  192
Length:  10; Size in bytes:  192
Length:  11; Size in bytes:  192
Length:  12; Size in bytes:  192
Length:  13; Size in bytes:  192
Length:  14; Size in bytes:  192
Length:  15; Size in bytes:  192
Length:  16; Size in bytes:  192
Length:  17; Size in bytes:  264
Length:  18; Size in bytes:  264
Length:  19; Size in bytes:  264


### 5.3.1 Implementing A Dynamic Array

If an element is appended to a list at a time when the array is full, we perform following steps:

1. Allocate a new array B with larger capacity.
2. Set B[i] = A[i], for i = 0,..., n-1, where n denotes current number of items.
3. Set A = B, that is, we use B as the array supporting the list.
4. Insert the new element in the new array.

Let's implement a dynamic array as a class:

In [4]:
import ctypes

class DynamicArray:
    """A dynamic array class akin to simplified Python list."""
    
    def __init__(self):
        """Create an empty array"""
        self._n = 0
        self._capacity = 1
        # Low level array
        self._A = self._make_array(self._capacity)
        
    def __len__(self):
        """Return number of elements stored in the array."""
        return self._n
    
    def __getitem__(self, k):
        """Return element at index k."""
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        return self._A[k]
    
    def append(self, obj):
        """Add object to end of the array"""
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1
        
    def _resize(self, c):
        """Resize internal array to capacity c."""
        B = self._make_array(c)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c
        
    def _make_array(self, c):
        """Return new array with capacity c"""
        return (c * ctypes.py_object)()

### 5.3.2 Amortized Analysis of Dynamic Arrays

